In [ ]:
import pandas as pd, numpy as np
import seaborn as sns

In [ ]:
from sklearn.model_selection import GroupShuffleSplit,StratifiedShuffleSplit

In [ ]:
final_data=dict()

for k in ['hk','igg']:
    data=pd.read_pickle("../../data/dsp_data_expr.pkl")[k]

    data['pheno']['sex']=(data['pheno']['sex']=="M").astype(float)
    any_mets=data['pheno']['any_mets']
    batch=data['pheno']['batch']
    macro_annot=data['pheno']['macro_annot']
    macro_annot_coded=pd.get_dummies(macro_annot)[['intra','inter']]
    macro_annot=macro_annot.map(dict(intra=0,inter=1,away=2))
    X=pd.concat([data['expr'],data['pheno'][['age','sex']],macro_annot_coded],axis=1)
    data['pheno']=pd.concat([data['pheno'],macro_annot_coded],axis=1)
    patient=data['pheno']['patient_id']


    data['cv_splits_mets']=pd.DataFrame(index=data['pheno'].index)
    data['cv_splits_macro']=pd.DataFrame(index=data['pheno'].index)
    data['cv_splits_macro_batch']=pd.DataFrame(index=data['pheno'].index)
    
    data['xy']=data['pheno'][['roi_x','roi_y']]

    ss=GroupShuffleSplit(n_splits=10, random_state=42, test_size=None,
                train_size=0.7)

    for i,(train_idx,test_idx) in enumerate(list(ss.split(X=X,y=any_mets,groups=batch))):
        data['cv_splits_mets'][f"batch_{i}"]=""
        data['cv_splits_mets'].loc[data['pheno'].index[train_idx],f"batch_{i}"]="train"
        data['cv_splits_mets'].loc[data['pheno'].index[test_idx],f"batch_{i}"]="test"

    for i,(train_idx,test_idx) in enumerate(list(ss.split(X=X,y=macro_annot,groups=patient))):
        data['cv_splits_macro'][f"batch_{i}"]=""
        data['cv_splits_macro'].loc[data['pheno'].index[train_idx],f"batch_{i}"]="train"
        data['cv_splits_macro'].loc[data['pheno'].index[test_idx],f"batch_{i}"]="test"

    for i,(train_idx,test_idx) in enumerate(list(ss.split(X=X,y=macro_annot,groups=batch))):
        data['cv_splits_macro_batch'][f"batch_{i}"]=""
        data['cv_splits_macro_batch'].loc[data['pheno'].index[train_idx],f"batch_{i}"]="train"
        data['cv_splits_macro_batch'].loc[data['pheno'].index[test_idx],f"batch_{i}"]="test"

    ss=StratifiedShuffleSplit(n_splits=10, random_state=42, test_size=None,
                train_size=0.7)
    data['cv_splits_mets_stratified_shuffle']=pd.DataFrame(index=data['pheno'].index)
    data['cv_splits_macro_stratified_shuffle']=pd.DataFrame(index=data['pheno'].index)
    data['cv_splits_macro_batch_stratified_shuffle']=pd.DataFrame(index=data['pheno'].index)

    for i,(train_idx,test_idx) in enumerate(list(ss.split(X=X,y=any_mets,groups=batch))):
        data['cv_splits_mets_stratified_shuffle'][f"batch_{i}"]=""
        data['cv_splits_mets_stratified_shuffle'].loc[data['pheno'].index[train_idx],f"batch_{i}"]="train"
        data['cv_splits_mets_stratified_shuffle'].loc[data['pheno'].index[test_idx],f"batch_{i}"]="test"

    for i,(train_idx,test_idx) in enumerate(list(ss.split(X=X,y=macro_annot,groups=patient))):
        data['cv_splits_macro_stratified_shuffle'][f"batch_{i}"]=""
        data['cv_splits_macro_stratified_shuffle'].loc[data['pheno'].index[train_idx],f"batch_{i}"]="train"
        data['cv_splits_macro_stratified_shuffle'].loc[data['pheno'].index[test_idx],f"batch_{i}"]="test"

    for i,(train_idx,test_idx) in enumerate(list(ss.split(X=X,y=macro_annot,groups=batch))):
        data['cv_splits_macro_batch_stratified_shuffle'][f"batch_{i}"]=""
        data['cv_splits_macro_batch_stratified_shuffle'].loc[data['pheno'].index[train_idx],f"batch_{i}"]="train"
        data['cv_splits_macro_batch_stratified_shuffle'].loc[data['pheno'].index[test_idx],f"batch_{i}"]="test"
    final_data[k]=data

In [ ]:
import pickle
pickle.dump(final_data,open('../../data/cv_split_data_final.pkl','wb'))
# final touches on datasets (e.g., scaling of age) produced final_splits